In [1]:
import json
from pathlib import Path

from arcgis.features import GeoAccessor
from arcgis.geometry import Geometry
import arcpy
import pandas as pd

In [2]:
# paths to common data locations - NOTE: to convert any path to a raw string, simply use str(path_instance)
dir_prj = Path.cwd().parent
dir_data = dir_prj/'data'
dir_raw = dir_data/'raw'
dir_int = dir_data/'interim'
gdb_int = dir_int/'interim.gdb'

# create the interim geodatabase if arcpy is available and it does not already exist
if not arcpy.Exists(str(gdb_int)):
    arcpy.management.CreateFileGDB(str(gdb_int.parent), gdb_int.name)

orig_fc = r'D:\projects\olympia-crime\data\interim\interim.gdb\olympia_h3_l09'

orig_fc

'D:\\projects\\olympia-crime\\data\\interim\\interim.gdb\\olympia_h3_l09'

In [21]:
orig_df = GeoAccessor.from_featureclass(orig_fc).drop(columns='OBJECTID').spatial.set_geometry('SHAPE', inplace=False)
orig_df.SHAPE = orig_df.SHAPE.apply(lambda geom: json.dumps(geom))
orig_df

,h3_id_str,h3_id,SHAPE
0,8928d590297ffff,617711512338300927,"{""rings"": [[[-122.94740574999997, 47.024184694..."
1,8928d5902b3ffff,617711512340135935,"{""rings"": [[[-122.95158429099996, 47.028134405..."
2,8928d591533ffff,617711512650514431,"{""rings"": [[[-122.92062630199996, 47.028123088..."
3,8928d5916afffff,617711512675418111,"{""rings"": [[[-122.87055727599994, 47.031318454..."
4,8928d591433ffff,617711512633737215,"{""rings"": [[[-122.89294779499994, 47.020566091..."
...,...,...,...
620,8928d591c6bffff,617711512771624959,"{""rings"": [[[-122.93495716799998, 47.062941352..."
621,8928d5913c3ffff,617711512626397183,"{""rings"": [[[-122.88002538999996, 47.053582475..."
622,8928d5918d7ffff,617711512711593983,"{""rings"": [[[-122.93913735299998, 47.066889033..."
623,8928d59165bffff,617711512669913087,"{""rings"": [[[-122.84353807499997, 47.032367414..."


In [22]:
orig_df.to_csv(dir_int/'h3_test.csv')

In [8]:
h3_df = pd.read_csv(dir_int/'h3_test.csv', index_col=0)
h3_df.SHAPE = h3_df.SHAPE.apply(lambda geom: Geometry(json.loads(geom)))
h3_df.h3_id = h3_df.h3_id.astype(str)
h3_df.spatial.set_geometry('SHAPE')

h3_df.info()
h3_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 625 entries, 0 to 624
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   h3_id_str  625 non-null    object  
 1   h3_id      625 non-null    object  
 2   SHAPE      625 non-null    geometry
dtypes: geometry(1), object(2)
memory usage: 19.5+ KB


,h3_id_str,h3_id,SHAPE
0,8928d590297ffff,617711512338300927,"{""rings"": [[[-122.94740574999997, 47.024184694..."
1,8928d5902b3ffff,617711512340135935,"{""rings"": [[[-122.95158429099996, 47.028134405..."
2,8928d591533ffff,617711512650514431,"{""rings"": [[[-122.92062630199996, 47.028123088..."
3,8928d5916afffff,617711512675418111,"{""rings"": [[[-122.87055727599994, 47.031318454..."
4,8928d591433ffff,617711512633737215,"{""rings"": [[[-122.89294779499994, 47.020566091..."


In [9]:
h3_fc = h3_df.spatial.to_featureclass(f'{gdb_int}/oly_h3')

h3_fc

'D:\\projects\\geoenrichment-samples\\Python\\geoenrichment-introspection\\data\\interim\\interim.gdb\\oly_h3'